In [ ]:
!pip install pandas openai scikit-learn nltk rouge-score bert-score torch deepeval spacy rouge

In [ ]:
!pip install --upgrade deepeval

In [ ]:
#subset_df.shape

In [1]:
import pandas as pd
import openai
from nltk.translate.bleu_score import sentence_bleu, SmoothingFunction
from rouge import Rouge
from bert_score import score
import torch
from torch.nn import CrossEntropyLoss
import warnings
import os

# Suppress warnings
warnings.filterwarnings('ignore')

# ✅ DeepEval imports
from deepeval.metrics import FaithfulnessMetric, AnswerRelevancyMetric, GEval
from deepeval.test_case import LLMTestCase


A module that was compiled using NumPy 1.x cannot be run in
NumPy 2.2.4 as it may crash. To support both 1.x and 2.x
versions of NumPy, modules must be compiled with NumPy 2.0.
Some module may need to rebuild instead e.g. with 'pybind11>=2.12'.

If you are a user of the module, the easiest solution will be to
downgrade to 'numpy<2' or try to upgrade the affected module.
We expect that some modules will need time to support NumPy 2.

Traceback (most recent call last):  File "/usr/lib/python3.10/runpy.py", line 196, in _run_module_as_main
    return _run_code(code, main_globals, None,
  File "/usr/lib/python3.10/runpy.py", line 86, in _run_code
    exec(code, run_globals)
  File "/usr/local/lib/python3.10/dist-packages/ipykernel_launcher.py", line 17, in <module>
    app.launch_new_instance()
  File "/usr/local/lib/python3.10/dist-packages/traitlets/config/application.py", line 1053, in launch_instance
    app.start()
  File "/usr/local/lib/python3.10/dist-packages/ipykernel/kernelapp.p

In [2]:
df = pd.read_csv("dolly.csv")
subset_df = df.sample(n=min(100, len(df)), random_state=42)

In [3]:
df.shape

(30, 3)

In [4]:
df= df.tail(3)

In [5]:
df

,instruction,context,response
15,"Who coined the phrase ""Bike-shedding"" and when?",The law of triviality is C. Northcote Parkinso...,"The phrase ""bike-shedding"" was introduced in 1..."
16,Given a reference text about Run Towards the D...,Run Towards the Danger is a 2022 Canadian essa...,Six essays are part of the Run Towards the Dan...
17,Who tends to participates in hackathons?,"A hackathon (also known as a hack day, hackfes...",Computer programmers and others involved in so...
18,What is Marketing Automation?,Marketing Automation is a subset of customer r...,Marketing Automation is software that helps ma...
19,Which American president spoke Dutch as their ...,Van Buren received a basic education at the vi...,President Van Buren spoke Dutch as his first l...
20,Given this reference text about the consumptio...,The United States is a leading exporter of pea...,National Peanut Butter Day takes place on Janu...
21,What does fasting mean?,Fasting is the abstention from eating and some...,Fasting is the abstention from eating and some...
22,Was Furze Hill an established community in the...,Furze Hill (or Furzehill) is a hamlet situated...,"No, it was not an established community until ..."
23,What is wine?,Wine is an alcoholic drink typically made from...,Wine is an alcoholic drink typically made from...
24,What is the last festival celebrated in Tihar ...,"Lakshmi Puja is celebrated as a part of Tihar,...",The festival lasts for 5 days and Bhai Tika is...


In [7]:
##computing deepeval metrics

In [8]:
import pandas as pd
import openai
from deepeval.metrics import FaithfulnessMetric, AnswerRelevancyMetric
from deepeval.test_case import LLMTestCase

# Set OpenAI API Key

# Load Dataset
#df = pd.read_csv("dolly.csv")
subset_df = df.sample(n=min(100, len(df)), random_state=42)





# ✅ Define GPT-4o QA Function
def answer_with_gpt4o(context):
    prompt_text = f"""
    Based on the context below, generate a concise and accurate response that answers the implied question.

    Context: "{context}"

    Answer:
    """
    try:
        response = openai.chat.completions.create(
            model="gpt-4o",
            messages=[{"role": "user", "content": prompt_text}],
            temperature=0.2,
            max_tokens=200
        )
        return response.choices[0].message.content.strip()
    except Exception as e:
        print(f"Error during QA generation: {e}")
        return None  # Make sure this is aligned with the try-block

# ✅ Generate Predictions (now properly unindented)
subset_df['predicted_response'] = subset_df['context'].apply(answer_with_gpt4o)








# Corrected Test Case Creation
test_cases = []
for _, row in subset_df.iterrows():
    test_cases.append(LLMTestCase(
        input=row['instruction'],  # Use the INSTRUCTION as input
        actual_output=row['predicted_response'],
        expected_output=row['response'],
        retrieval_context=[row['context']]  # Provide context for faithfulness
    ))

# Initialize Metrics
faithfulness_metric = FaithfulnessMetric(threshold=0.5)
relevancy_metric = AnswerRelevancyMetric(threshold=0.5)

# Evaluate Metrics
faithfulness_scores = []
relevancy_scores = []

for test_case in test_cases:
    # Measure Faithfulness
    faithfulness_metric.measure(test_case)
    faithfulness_scores.append(faithfulness_metric.score)
    
    # Measure Relevancy
    relevancy_metric.measure(test_case)
    relevancy_scores.append(relevancy_metric.score)

# Calculate averages
avg_faithfulness = sum(faithfulness_scores) / len(faithfulness_scores)
avg_relevancy = sum(relevancy_scores) / len(relevancy_scores)

print(f"Average Faithfulness Score: {avg_faithfulness:.2%}")
print(f"Average Answer Relevancy Score: {avg_relevancy:.2%}")

Output()

Output()

Output()

Output()

Output()

...

Output()

Output()

Output()

Output()

Average Faithfulness Score: 94.11%
Average Answer Relevancy Score: 53.10%


In [9]:
# ... individusual scoring..

# Evaluate Metrics and store in lists
faithfulness_scores = []
relevancy_scores = []

for test_case in test_cases:
    # Measure Faithfulness
    faithfulness_metric.measure(test_case)
    faithfulness_scores.append(faithfulness_metric.score)
    
    # Measure Relevancy
    relevancy_metric.measure(test_case)
    relevancy_scores.append(relevancy_metric.score)

# Add scores directly to the DataFrame
subset_df['faithfulness_score'] = faithfulness_scores
subset_df['relevancy_score'] = relevancy_scores

# Create new DataFrame with all required columns (this is already done in-place)
# If you need to save it:
final_deepeval_df = subset_df[['instruction', 'context', 'response', 
                     'predicted_response', 'faithfulness_score', 
                     'relevancy_score']].copy()

# Optional: Save to CSV
final_deepeval_df.to_csv('Deep_Eval_evaluated_responses.csv', index=False)



Output()

Output()

Output()

Output()

Output()

...

Output()

Output()

Output()

Output()

Output()

In [10]:
final_deepeval_df

,instruction,context,response,predicted_response,faithfulness_score,relevancy_score
24,What is the last festival celebrated in Tihar ...,"Lakshmi Puja is celebrated as a part of Tihar,...",The festival lasts for 5 days and Bhai Tika is...,Lakshmi Puja is celebrated on the third day of...,0.5,0.222222
26,"Based on this paragraph, how many younger sibl...","Kennedy was born on February 22, 1932, at St. ...","Based on this paragraph, Kennedy had no younge...","The youngest Kennedy child, born on February 2...",1.0,0.333333
15,"Who coined the phrase ""Bike-shedding"" and when?",The law of triviality is C. Northcote Parkinso...,"The phrase ""bike-shedding"" was introduced in 1...","The law of triviality, also known as the bike-...",1.0,0.000000
28,"Given a reference text about Jen Shah, what is...",Shah starred on the reality television series ...,Jen Shah came into the spotlight in November 2...,"Jen Shah, a star of The Real Housewives of Sal...",1.0,0.857143
20,Given this reference text about the consumptio...,The United States is a leading exporter of pea...,National Peanut Butter Day takes place on Janu...,The United States is a major exporter and cons...,1.0,0.000000
23,What is wine?,Wine is an alcoholic drink typically made from...,Wine is an alcoholic drink typically made from...,"Wine is primarily made from fermented grapes, ...",1.0,1.000000
17,Who tends to participates in hackathons?,"A hackathon (also known as a hack day, hackfes...",Computer programmers and others involved in so...,A hackathon is an event where participants eng...,1.0,0.428571
16,Given a reference text about Run Towards the D...,Run Towards the Danger is a 2022 Canadian essa...,Six essays are part of the Run Towards the Dan...,"""Run Towards the Danger"" is a 2022 essay colle...",1.0,0.000000
29,What types of problems in government could AI ...,The potential uses of AI in government are wid...,The potential uses of AI in government are wid...,AI can be effectively utilized in government o...,1.0,1.000000
19,Which American president spoke Dutch as their ...,Van Buren received a basic education at the vi...,President Van Buren spoke Dutch as his first l...,"Martin Van Buren, the only U.S. president whos...",1.0,0.375000


In [11]:
###combined  individual and average
import pandas as pd
import openai
from deepeval.metrics import FaithfulnessMetric, AnswerRelevancyMetric
from deepeval.test_case import LLMTestCase

# Set OpenAI API Key
# openai.api_key = 'your-api-key'

# Load Dataset
# df = pd.read_csv("dolly.csv")
subset_df = df.sample(n=min(100, len(df)), random_state=42)

# ✅ Define GPT-4o QA Function
def answer_with_gpt4o(context):
    prompt_text = f"""
    Based on the context below, generate a concise and accurate response that answers the implied question.

    Context: "{context}"

    Answer:
    """
    try:
        response = openai.chat.completions.create(
            model="gpt-4o",
            messages=[{"role": "user", "content": prompt_text}],
            temperature=0.2,
            max_tokens=200
        )
        return response.choices[0].message.content.strip()
    except Exception as e:
        print(f"Error during QA generation: {e}")
        return None

# Generate Predictions
subset_df['predicted_response'] = subset_df['context'].apply(answer_with_gpt4o)

# Initialize empty lists to store scores for each row
faithfulness_scores = []
relevancy_scores = []

# Create a list to hold the hallu_deepeval data
hallu_deepeval_rows = []

for _, row in subset_df.iterrows():
    # Create test case
    test_case = LLMTestCase(
        input=row['instruction'],
        actual_output=row['predicted_response'],
        expected_output=row['response'],
        retrieval_context=[row['context']]
    )
    # Measure metrics
    faithfulness_metric = FaithfulnessMetric(threshold=0.5)
    relevancy_metric = AnswerRelevancyMetric(threshold=0.5)
    
    faithfulness_metric.measure(test_case)
    relevancy_metric.measure(test_case)
    
    # Append scores
    faithfulness_scores.append(faithfulness_metric.score)
    relevancy_scores.append(relevancy_metric.score)
    
    # Append the row with additional scores
    hallu_deepeval_rows.append({
        **row,
        'faithfulness_score': faithfulness_metric.score,
        'relevancy_score': relevancy_metric.score
    })

# Create a new DataFrame from hallu_deepeval_rows
hallu_deepeval_df = pd.DataFrame(hallu_deepeval_rows)

# Now, hallu_deepeval_df contains all original data plus the predicted_response and scores
# You can save or print this DataFrame
print(hallu_deepeval_df.head())

# If you want, you can also compute overall averages
avg_faithfulness = sum(faithfulness_scores) / len(faithfulness_scores)
avg_relevancy = sum(relevancy_scores) / len(relevancy_scores)

print(f"Average Faithfulness Score: {avg_faithfulness:.2%}")
print(f"Average Answer Relevancy Score: {avg_relevancy:.2%}")

Output()

Output()

Output()

Output()

Output()

...

Output()

Output()

Output()

Output()

                                         instruction  \
0  What is the last festival celebrated in Tihar ...   
1  Based on this paragraph, how many younger sibl...   
2    Who coined the phrase "Bike-shedding" and when?   
3  Given a reference text about Jen Shah, what is...   
4  Given this reference text about the consumptio...   

                                             context  \
0  Lakshmi Puja is celebrated as a part of Tihar,...   
1  Kennedy was born on February 22, 1932, at St. ...   
2  The law of triviality is C. Northcote Parkinso...   
3  Shah starred on the reality television series ...   
4  The United States is a leading exporter of pea...   

                                            response  \
0  The festival lasts for 5 days and Bhai Tika is...   
1  Based on this paragraph, Kennedy had no younge...   
2  The phrase "bike-shedding" was introduced in 1...   
3  Jen Shah came into the spotlight in November 2...   
4  National Peanut Butter Day takes place on J

In [12]:
hallu_deepeval_df.to_csv('Hallu_deepeval_df_responses.csv', index=False)


In [13]:
import os
os.environ["OPENAI_API_KEY"] = ""

In [14]:
hallu_deepeval_df

,instruction,context,response,predicted_response,faithfulness_score,relevancy_score
0,What is the last festival celebrated in Tihar ...,"Lakshmi Puja is celebrated as a part of Tihar,...",The festival lasts for 5 days and Bhai Tika is...,Lakshmi Puja is celebrated on the third day of...,1.000000,0.200000
1,"Based on this paragraph, how many younger sibl...","Kennedy was born on February 22, 1932, at St. ...","Based on this paragraph, Kennedy had no younge...",Kennedy was named after his father's assistant...,1.000000,0.000000
2,"Who coined the phrase ""Bike-shedding"" and when?",The law of triviality is C. Northcote Parkinso...,"The phrase ""bike-shedding"" was introduced in 1...","The law of triviality, also known as the bike-...",1.000000,0.000000
3,"Given a reference text about Jen Shah, what is...",Shah starred on the reality television series ...,Jen Shah came into the spotlight in November 2...,"Jen Shah, a star of The Real Housewives of Sal...",1.000000,1.000000
4,Given this reference text about the consumptio...,The United States is a leading exporter of pea...,National Peanut Butter Day takes place on Janu...,The United States is a major exporter and cons...,1.000000,0.000000
5,What is wine?,Wine is an alcoholic drink typically made from...,Wine is an alcoholic drink typically made from...,"Wine is primarily made from fermented grapes, ...",1.000000,0.545455
6,Who tends to participates in hackathons?,"A hackathon (also known as a hack day, hackfes...",Computer programmers and others involved in so...,A hackathon is an event where participants eng...,1.000000,0.428571
7,Given a reference text about Run Towards the D...,Run Towards the Danger is a 2022 Canadian essa...,Six essays are part of the Run Towards the Dan...,"""Run Towards the Danger"" is a 2022 essay colle...",1.000000,0.000000
8,What types of problems in government could AI ...,The potential uses of AI in government are wid...,The potential uses of AI in government are wid...,AI can be effectively applied in government op...,1.000000,1.000000
9,Which American president spoke Dutch as their ...,Van Buren received a basic education at the vi...,President Van Buren spoke Dutch as his first l...,"Martin Van Buren, the only U.S. president whos...",1.000000,0.428571


In [15]:
print(subset_df[subset_df['context'].isnull()])

Empty DataFrame
Columns: [instruction, context, response, predicted_response]
Index: []


In [16]:
subset_df

,instruction,context,response,predicted_response
24,What is the last festival celebrated in Tihar ...,"Lakshmi Puja is celebrated as a part of Tihar,...",The festival lasts for 5 days and Bhai Tika is...,Lakshmi Puja is celebrated on the third day of...
26,"Based on this paragraph, how many younger sibl...","Kennedy was born on February 22, 1932, at St. ...","Based on this paragraph, Kennedy had no younge...",Kennedy was named after his father's assistant...
15,"Who coined the phrase ""Bike-shedding"" and when?",The law of triviality is C. Northcote Parkinso...,"The phrase ""bike-shedding"" was introduced in 1...","The law of triviality, also known as the bike-..."
28,"Given a reference text about Jen Shah, what is...",Shah starred on the reality television series ...,Jen Shah came into the spotlight in November 2...,"Jen Shah, a star of The Real Housewives of Sal..."
20,Given this reference text about the consumptio...,The United States is a leading exporter of pea...,National Peanut Butter Day takes place on Janu...,The United States is a major exporter and cons...
23,What is wine?,Wine is an alcoholic drink typically made from...,Wine is an alcoholic drink typically made from...,"Wine is primarily made from fermented grapes, ..."
17,Who tends to participates in hackathons?,"A hackathon (also known as a hack day, hackfes...",Computer programmers and others involved in so...,A hackathon is an event where participants eng...
16,Given a reference text about Run Towards the D...,Run Towards the Danger is a 2022 Canadian essa...,Six essays are part of the Run Towards the Dan...,"""Run Towards the Danger"" is a 2022 essay colle..."
29,What types of problems in government could AI ...,The potential uses of AI in government are wid...,The potential uses of AI in government are wid...,AI can be effectively applied in government op...
19,Which American president spoke Dutch as their ...,Van Buren received a basic education at the vi...,President Van Buren spoke Dutch as his first l...,"Martin Van Buren, the only U.S. president whos..."


In [17]:
## convert context to a list of string
test_cases = []
for _, row in subset_df.iterrows():
    test_cases.append(LLMTestCase(
        input=row['instruction'],
        actual_output=row['predicted_response'],
        expected_output=row['response'],
        context=[row['context']],            # <-- list of strings here!
        retrieval_context=[row['context']]   # as before
    ))

In [18]:
## average score
from deepeval.metrics import HallucinationMetric

hallucination_metric = HallucinationMetric(threshold=0.5)

hallucination_scores = []

for test_case in test_cases:
    try:
        hallucination_metric.measure(test_case)
        hallucination_scores.append(hallucination_metric.score)
    except Exception as e:
        print(f"Error measuring hallucination for test case: {e}")
        hallucination_scores.append(0.0)

if hallucination_scores:
    avg_hallucination = sum(hallucination_scores) / len(hallucination_scores)
    print(f"Average Hallucination Score: {avg_hallucination:.2%}")
else:
    print("No hallucination scores computed.")

Output()

Output()

Output()

Output()

Output()

...

Output()

Output()

Output()

Output()

Average Hallucination Score: 0.00%


In [19]:
##average and inidvuao scoes
from deepeval.metrics import HallucinationMetric

# Initialize hallucination metric
hallucination_metric = HallucinationMetric(threshold=0.5)

# Prepare list for hallucination scores
hallucination_scores = []

# Prepare list to hold hallu_deepeval rows
hallu_deepeval_rows = []

for _, row in subset_df.iterrows():
    # Create test case
    test_case = LLMTestCase(
        input=row['instruction'],
        actual_output=row['predicted_response'],
        expected_output=row['response'],
        retrieval_context=[row['context']]
    )
    # Measure existing metrics
    faithfulness_metric = FaithfulnessMetric(threshold=0.5)
    relevancy_metric = AnswerRelevancyMetric(threshold=0.5)

    faithfulness_metric.measure(test_case)
    relevancy_metric.measure(test_case)

    # Measure hallucination
    try:
        hallucination_metric.measure(test_case)
        hallucination_score = hallucination_metric.score
    except Exception as e:
        print(f"Error measuring hallucination for row: {e}")
        hallucination_score = 0.0

    # Append hallucination score
    hallucination_scores.append(hallucination_score)

    # Append all data + scores
    hallu_deepeval_rows.append({
        **row,
        'faithfulness_score': faithfulness_metric.score,
        'relevancy_score': relevancy_metric.score,
        'hallucination_score': hallucination_score
    })

# Create a new DataFrame with all scores
hallu_deepeval_df = pd.DataFrame(hallu_deepeval_rows)

# Print or save the hallu_deepeval DataFrame
print(hallu_deepeval_df.head())

# Calculate overall average hallucination score
if hallucination_scores:
    avg_hallucination = sum(hallucination_scores) / len(hallucination_scores)
    print(f"Average Hallucination Score: {avg_hallucination:.2%}")
else:
    print("No hallucination scores computed.")

Output()

Output()

Output()

Error measuring hallucination for row: 'context' cannot be None for the 'Hallucination' metric


Output()

...

Output()

Output()

Error measuring hallucination for row: 'context' cannot be None for the 'Hallucination' metric


Output()

Error measuring hallucination for row: 'context' cannot be None for the 'Hallucination' metric
                                         instruction  \
0  What is the last festival celebrated in Tihar ...   
1  Based on this paragraph, how many younger sibl...   
2    Who coined the phrase "Bike-shedding" and when?   
3  Given a reference text about Jen Shah, what is...   
4  Given this reference text about the consumptio...   

                                             context  \
0  Lakshmi Puja is celebrated as a part of Tihar,...   
1  Kennedy was born on February 22, 1932, at St. ...   
2  The law of triviality is C. Northcote Parkinso...   
3  Shah starred on the reality television series ...   
4  The United States is a leading exporter of pea...   

                                            response  \
0  The festival lasts for 5 days and Bhai Tika is...   
1  Based on this paragraph, Kennedy had no younge...   
2  The phrase "bike-shedding" was introduced in 1...   
3  Jen 

In [20]:
hallu_deepeval_df.to_csv('Withhallu_deepeval_df_responses.csv', index=False)


In [ ]:
##genericmetrics

In [21]:
!pip install spacy


[notice] A new release of pip is available: 23.3.1 -> 25.0.1
[notice] To update, run: python -m pip install --upgrade pip


In [22]:
import pandas as pd
import openai
import spacy
from deepeval.metrics import FaithfulnessMetric, AnswerRelevancyMetric, HallucinationMetric
from deepeval.test_case import LLMTestCase
from nltk.translate.bleu_score import sentence_bleu
from rouge import Rouge
from bert_score import score
import numpy as np

# Install required packages if missing (run once)
# !pip install nltk rouge spacy bert-score python-Levenshtein
# !python -m spacy download en_core_web_sm
#import en_core_web_sm

#nlp = en_core_web_sm.load()
# Initialize NLP models
nlp = spacy.load("en_core_web_sm")
rouge = Rouge()


In [23]:
!python -m spacy download en_core_web_sm


A module that was compiled using NumPy 1.x cannot be run in
NumPy 2.2.4 as it may crash. To support both 1.x and 2.x
versions of NumPy, modules must be compiled with NumPy 2.0.
Some module may need to rebuild instead e.g. with 'pybind11>=2.12'.

If you are a user of the module, the easiest solution will be to
downgrade to 'numpy<2' or try to upgrade the affected module.
We expect that some modules will need time to support NumPy 2.

Traceback (most recent call last):  File "/usr/lib/python3.10/runpy.py", line 187, in _run_module_as_main
    mod_name, mod_spec, code = _get_module_details(mod_name, _Error)
  File "/usr/lib/python3.10/runpy.py", line 146, in _get_module_details
    return _get_module_details(pkg_main_name, error)
  File "/usr/lib/python3.10/runpy.py", line 110, in _get_module_details
    __import__(pkg_name)
  File "/usr/local/lib/python3.10/dist-packages/spacy/__init__.py", line 6, in <module>
    from .errors import setup_default_warnings
  File "/usr/local/lib/python3

In [24]:
import spacy
from rouge import Rouge  # Make sure this import is correct
import pandas as pd
from nltk.translate.bleu_score import sentence_bleu
from bert_score import score

class StandardMetricsCalculator:
    def __init__(self):
        print("Initializing StandardMetricsCalculator...")  # Add print statement for debugging
        self.nlp = spacy.load("en_core_web_sm")
        # --- THIS IS THE CORRECTED PART for rouge library ---
        # REMOVED max_n=3, ADDED 'rouge-3' to metrics
        self.rouge = Rouge(metrics=['rouge-l', 'rouge-3'])
        print("...Rouge object initialized.")  # Add print statement

    def _calculate_exact_match(self, row):
        pred = str(row['predicted_response']).strip().lower()
        truth = str(row['response']).strip().lower()
        return 1 if pred == truth else 0

    def _calculate_f1(self, row):
        pred_tokens = set(str(row['predicted_response']).lower().split())
        truth_tokens = set(str(row['response']).lower().split())
        common = pred_tokens & truth_tokens
        if not pred_tokens or not truth_tokens:
            return 0.0
        precision = len(common)/len(pred_tokens)
        recall = len(common)/len(truth_tokens)
        return 2*(precision*recall)/(precision+recall) if (precision+recall) else 0

    def _calculate_bleu(self, row):
        try:
            return sentence_bleu(
                [str(row['response']).split()],
                str(row['predicted_response']).split(),
                weights=(0.5, 0.5, 0, 0)
            )
        except:
            return 0.0

    # --- Make sure _calculate_rouge uses self.rouge ---
    def _calculate_rouge(self, row):
        try:
            # Using the 'rouge' library's get_scores method
            scores = self.rouge.get_scores(
                hyps=str(row['predicted_response']),
                refs=str(row['response'])
            )
            # Check if scores were returned (can be empty for empty strings)
            if scores:
                return {
                    'rouge_l': scores[0]['rouge-l']['f'],
                    'rouge_3': scores[0]['rouge-3']['f']
                }
            else:
                return {'rouge_l': 0.0, 'rouge_3': 0.0}
        except Exception as e:
            # print(f"Error calculating ROUGE for row: {e}")  # Optional: print errors per row
            return {'rouge_l': 0.0, 'rouge_3': 0.0}

    def _calculate_keyword_overlap(self, row):
        pred_doc = self.nlp(str(row['predicted_response']))
        truth_doc = self.nlp(str(row['response']))
        pred_keys = {t.lemma_ for t in pred_doc if t.pos_ in ['NOUN','PROPN','VERB']}
        truth_keys = {t.lemma_ for t in truth_doc if t.pos_ in ['NOUN','PROPN','VERB']}
        return len(pred_keys & truth_keys)/len(truth_keys) if truth_keys else 0.0

    def compute_metrics(self, df):
        metrics_df = df.apply(lambda row: pd.Series({
            'accuracy': self._calculate_exact_match(row),
            'f1_score': self._calculate_f1(row),
            'bleu_score': self._calculate_bleu(row),
            **self._calculate_rouge(row),
            'keyword_overlap': self._calculate_keyword_overlap(row),
            'length_ratio': len(str(row['predicted_response']).split()) / 
                           max(1, len(str(row['response']).split()))
        }), axis=1)
        
        # Add BERTScore separately (batch computation)
        _, _, bert_scores = score(
            df['predicted_response'].astype(str).tolist(),
            df['response'].astype(str).tolist(),
            lang='en',
            model_type='bert-base-uncased'
        )
        # Convert bert_scores to a list of floats
        metrics_df['bert_score'] = [score.item() for score in bert_scores]
        
        return metrics_df

# Initialize calculator
metric_calculator = StandardMetricsCalculator()

# Compute standard metrics (returns new DF with only metric columns)
standard_metrics_df = metric_calculator.compute_metrics(subset_df)

# Combine with original DF horizontally
final_df = pd.concat([subset_df, standard_metrics_df], axis=1)

# Keep original column order + new metrics
final_columns = [
    'instruction', 'context', 'response', 'predicted_response',
    'accuracy', 'f1_score', 'bleu_score', 'rouge_l', 'rouge_3',
    'bert_score', 'keyword_overlap', 'length_ratio'
]

final_df = final_df[final_columns].copy()

Initializing StandardMetricsCalculator...
...Rouge object initialized.


In [25]:
standard_metrics_df 

,accuracy,f1_score,bleu_score,rouge_l,rouge_3,keyword_overlap,length_ratio,bert_score
24,0.0,0.181818,5.273843e-155,0.121212,0.000000,0.200000,1.846154,0.567486
26,0.0,0.127660,4.430915e-155,0.085106,0.000000,0.166667,2.000000,0.502016
15,0.0,0.047619,2.274773e-155,0.047059,0.000000,0.142857,7.818182,0.432627
28,0.0,0.551724,2.660100e-01,0.511111,0.192308,0.653846,0.573529,0.730172
20,0.0,0.148148,3.515896e-155,0.140351,0.000000,0.200000,4.153846,0.451536
23,0.0,0.188679,7.239537e-02,0.185185,0.033333,0.800000,5.400000,0.603000
17,0.0,0.306122,1.259000e-01,0.323232,0.035398,0.461538,1.294118,0.650237
16,0.0,0.163265,5.025189e-02,0.163265,0.000000,0.400000,3.750000,0.505760
29,0.0,0.214286,5.690117e-02,0.144578,0.000000,0.473684,1.146341,0.634866
19,0.0,0.238806,7.476962e-02,0.257143,0.000000,0.454545,2.000000,0.566347


In [26]:
hallu_deepeval_df

,instruction,context,response,predicted_response,faithfulness_score,relevancy_score,hallucination_score
0,What is the last festival celebrated in Tihar ...,"Lakshmi Puja is celebrated as a part of Tihar,...",The festival lasts for 5 days and Bhai Tika is...,Lakshmi Puja is celebrated on the third day of...,0.666667,0.000000,0.0
1,"Based on this paragraph, how many younger sibl...","Kennedy was born on February 22, 1932, at St. ...","Based on this paragraph, Kennedy had no younge...",Kennedy was named after his father's assistant...,1.000000,0.000000,0.0
2,"Who coined the phrase ""Bike-shedding"" and when?",The law of triviality is C. Northcote Parkinso...,"The phrase ""bike-shedding"" was introduced in 1...","The law of triviality, also known as the bike-...",1.000000,0.000000,0.0
3,"Given a reference text about Jen Shah, what is...",Shah starred on the reality television series ...,Jen Shah came into the spotlight in November 2...,"Jen Shah, a star of The Real Housewives of Sal...",1.000000,0.800000,0.0
4,Given this reference text about the consumptio...,The United States is a leading exporter of pea...,National Peanut Butter Day takes place on Janu...,The United States is a major exporter and cons...,1.000000,0.000000,0.0
5,What is wine?,Wine is an alcoholic drink typically made from...,Wine is an alcoholic drink typically made from...,"Wine is primarily made from fermented grapes, ...",1.000000,1.000000,0.0
6,Who tends to participates in hackathons?,"A hackathon (also known as a hack day, hackfes...",Computer programmers and others involved in so...,A hackathon is an event where participants eng...,1.000000,0.428571,0.0
7,Given a reference text about Run Towards the D...,Run Towards the Danger is a 2022 Canadian essa...,Six essays are part of the Run Towards the Dan...,"""Run Towards the Danger"" is a 2022 essay colle...",1.000000,0.000000,0.0
8,What types of problems in government could AI ...,The potential uses of AI in government are wid...,The potential uses of AI in government are wid...,AI can be effectively applied in government op...,1.000000,1.000000,0.0
9,Which American president spoke Dutch as their ...,Van Buren received a basic education at the vi...,President Van Buren spoke Dutch as his first l...,"Martin Van Buren, the only U.S. president whos...",1.000000,0.285714,0.0


In [27]:
import spacy
from rouge import Rouge  # Make sure this import is correct
import pandas as pd
from nltk.translate.bleu_score import sentence_bleu
from bert_score import score

class StandardMetricsCalculator:
    def __init__(self):
        print("Initializing StandardMetricsCalculator...")  # Add print statement for debugging
        self.nlp = spacy.load("en_core_web_sm")
        # --- THIS IS THE CORRECTED PART for rouge library ---
        # REMOVED max_n=3, ADDED 'rouge-3' to metrics
        self.rouge = Rouge(metrics=['rouge-l', 'rouge-3'])
        print("...Rouge object initialized.")  # Add print statement

    def _calculate_exact_match(self, row):
        pred = str(row['predicted_response']).strip().lower()
        truth = str(row['response']).strip().lower()
        return 1 if pred == truth else 0

    def _calculate_f1(self, row):
        pred_tokens = set(str(row['predicted_response']).lower().split())
        truth_tokens = set(str(row['response']).lower().split())
        common = pred_tokens & truth_tokens
        if not pred_tokens or not truth_tokens:
            return 0.0
        precision = len(common)/len(pred_tokens)
        recall = len(common)/len(truth_tokens)
        return 2*(precision*recall)/(precision+recall) if (precision+recall) else 0

    def _calculate_bleu(self, row):
        try:
            return sentence_bleu(
                [str(row['response']).split()],
                str(row['predicted_response']).split(),
                weights=(0.5, 0.5, 0, 0)
            )
        except:
            return 0.0

    # --- Make sure _calculate_rouge uses self.rouge ---
    def _calculate_rouge(self, row):
        try:
            # Using the 'rouge' library's get_scores method
            scores = self.rouge.get_scores(
                hyps=str(row['predicted_response']),
                refs=str(row['response'])
            )
            # Check if scores were returned (can be empty for empty strings)
            if scores:
                return {
                    'rouge_l': scores[0]['rouge-l']['f'],
                    'rouge_3': scores[0]['rouge-3']['f']
                }
            else:
                return {'rouge_l': 0.0, 'rouge_3': 0.0}
        except Exception as e:
            # print(f"Error calculating ROUGE for row: {e}")  # Optional: print errors per row
            return {'rouge_l': 0.0, 'rouge_3': 0.0}

    def _calculate_keyword_overlap(self, row):
        pred_doc = self.nlp(str(row['predicted_response']))
        truth_doc = self.nlp(str(row['response']))
        pred_keys = {t.lemma_ for t in pred_doc if t.pos_ in ['NOUN','PROPN','VERB']}
        truth_keys = {t.lemma_ for t in truth_doc if t.pos_ in ['NOUN','PROPN','VERB']}
        return len(pred_keys & truth_keys)/len(truth_keys) if truth_keys else 0.0

    def compute_metrics(self, df):
        # Create a copy of the original DataFrame to include in the output
        metrics_df = df.copy()  # Start with the original data

        # Calculate metrics and add them as new columns
        metrics_df['accuracy'] = df.apply(self._calculate_exact_match, axis=1)
        metrics_df['f1_score'] = df.apply(self._calculate_f1, axis=1)
        metrics_df['bleu_score'] = df.apply(self._calculate_bleu, axis=1)

        # Calculate ROUGE and add the columns
        rouge_scores = df.apply(self._calculate_rouge, axis=1, result_type='expand')
        metrics_df['rouge_l'] = rouge_scores['rouge_l']
        metrics_df['rouge_3'] = rouge_scores['rouge_3']

        metrics_df['keyword_overlap'] = df.apply(self._calculate_keyword_overlap, axis=1)
        metrics_df['length_ratio'] = df.apply(lambda row: len(str(row['predicted_response']).split()) /
                                               max(1, len(str(row['response']).split())), axis=1)

        # Add BERTScore separately (batch computation)
        _, _, bert_scores = score(
            df['predicted_response'].astype(str).tolist(),
            df['response'].astype(str).tolist(),
            lang='en',
            model_type='bert-base-uncased'
        )
        # Convert bert_scores to a list of floats
        metrics_df['bert_score'] = [score.item() for score in bert_scores]

        return metrics_df


# Initialize calculator
metric_calculator = StandardMetricsCalculator()

# Assuming 'subset_df' is your DataFrame
# Compute metrics and append to the original DataFrame
final_df = metric_calculator.compute_metrics(subset_df)

# (Optional) Reorder columns if you want a specific order
final_columns = [
    'instruction', 'context', 'response', 'predicted_response',
    'accuracy', 'f1_score', 'bleu_score', 'rouge_l', 'rouge_3',
    'bert_score', 'keyword_overlap', 'length_ratio'
]

# Ensure all columns in final_columns exist in final_df, if not, keep all the original columns
final_columns = [col for col in final_columns if col in final_df.columns] + [col for col in final_df.columns if col not in final_columns]

final_df2 = final_df[final_columns].copy()


print(final_df2.head())  # Display the first few rows to verify the structure

Initializing StandardMetricsCalculator...
...Rouge object initialized.
                                          instruction  \
24  What is the last festival celebrated in Tihar ...   
26  Based on this paragraph, how many younger sibl...   
15    Who coined the phrase "Bike-shedding" and when?   
28  Given a reference text about Jen Shah, what is...   
20  Given this reference text about the consumptio...   

                                              context  \
24  Lakshmi Puja is celebrated as a part of Tihar,...   
26  Kennedy was born on February 22, 1932, at St. ...   
15  The law of triviality is C. Northcote Parkinso...   
28  Shah starred on the reality television series ...   
20  The United States is a leading exporter of pea...   

                                             response  \
24  The festival lasts for 5 days and Bhai Tika is...   
26  Based on this paragraph, Kennedy had no younge...   
15  The phrase "bike-shedding" was introduced in 1...   
28  Jen Shah ca

In [28]:
import spacy
from rouge import Rouge
import pandas as pd
from nltk.translate.bleu_score import sentence_bleu
from bert_score import score

class StandardMetricsCalculator:
    def __init__(self):
        print("Initializing StandardMetricsCalculator...")
        self.nlp = spacy.load("en_core_web_sm")
        self.rouge = Rouge(metrics=['rouge-l', 'rouge-3'])
        print("...Rouge object initialized.")

    def _calculate_exact_match(self, row):
        """Calculate exact match between predicted and true response."""
        pred = str(row['predicted_response']).strip().lower()
        truth = str(row['response']).strip().lower()
        return 1 if pred == truth else 0

    def _calculate_f1(self, row):
        """Calculate F1 score based on token overlap."""
        pred_tokens = set(str(row['predicted_response']).lower().split())
        truth_tokens = set(str(row['response']).lower().split())
        common = pred_tokens & truth_tokens
        if not pred_tokens or not truth_tokens:
            return 0.0
        precision = len(common) / len(pred_tokens)
        recall = len(common) / len(truth_tokens)
        return 2 * (precision * recall) / (precision + recall) if (precision + recall) else 0

    def _calculate_bleu(self, row):
        """Calculate BLEU score."""
        try:
            return sentence_bleu(
                references=[str(row['response']).split()],
                hypothesis=str(row['predicted_response']).split(),
                weights=(0.5, 0.5, 0, 0)
            )
        except Exception as e:
            print(f"Error calculating BLEU: {e}")
            return 0.0

    def _calculate_rouge(self, row):
        """Calculate ROUGE scores."""
        try:
            scores = self.rouge.get_scores(
                hyps=str(row['predicted_response']),
                refs=str(row['response'])
            )
            if scores:
                return {
                    'rouge_l': scores[0]['rouge-l']['f'],
                    'rouge_3': scores[0]['rouge-3']['f']
                }
            else:
                return {'rouge_l': 0.0, 'rouge_3': 0.0}
        except Exception as e:
            print(f"Error calculating ROUGE: {e}")
            return {'rouge_l': 0.0, 'rouge_3': 0.0}

    def _calculate_keyword_overlap(self, row):
        """Calculate keyword overlap using SpaCy."""
        pred_doc = self.nlp(str(row['predicted_response']))
        truth_doc = self.nlp(str(row['response']))
        pred_keys = {t.lemma_ for t in pred_doc if t.pos_ in ['NOUN', 'PROPN', 'VERB']}
        truth_keys = {t.lemma_ for t in truth_doc if t.pos_ in ['NOUN', 'PROPN', 'VERB']}
        return len(pred_keys & truth_keys) / len(truth_keys) if truth_keys else 0.0

    def compute_metrics(self, df):
        """Compute all metrics and append them to the DataFrame."""
        metrics_df = df.copy()

        # Calculate metrics and add them as new columns
        metrics_df['exact_match'] = df.apply(self._calculate_exact_match, axis=1)
        metrics_df['f1_score'] = df.apply(self._calculate_f1, axis=1)
        metrics_df['bleu_score'] = df.apply(self._calculate_bleu, axis=1)

        # Calculate ROUGE scores
        rouge_scores = df.apply(self._calculate_rouge, axis=1, result_type='expand')
        metrics_df['rouge_l'] = rouge_scores['rouge_l']
        metrics_df['rouge_3'] = rouge_scores['rouge_3']

        metrics_df['keyword_overlap'] = df.apply(self._calculate_keyword_overlap, axis=1)
        metrics_df['length_ratio'] = df.apply(lambda row: len(str(row['predicted_response']).split()) /
                                              max(1, len(str(row['response']).split())), axis=1)

        # Calculate BERTScore
        _, _, bert_scores = score(
            metrics_df['predicted_response'].astype(str).tolist(),
            metrics_df['response'].astype(str).tolist(),
            lang='en',
            model_type='bert-base-uncased'
        )
        metrics_df['bert_score'] = [score.item() for score in bert_scores]

        return metrics_df


# Example usage
# Assuming 'subset_df' is your DataFrame containing 'instruction', 'context', 'response', and 'predicted_response' columns
# subset_df = pd.DataFrame({...})

# Initialize calculator
metric_calculator = StandardMetricsCalculator()

# Compute metrics and append to the original DataFrame
final_df = metric_calculator.compute_metrics(subset_df)

# Reorder columns for better presentation
final_columns = [
    'instruction', 'context', 'response', 'predicted_response',
    'exact_match', 'f1_score', 'bleu_score', 'rouge_l', 'rouge_3',
    'bert_score', 'keyword_overlap', 'length_ratio'
]
final_columns = [col for col in final_columns if col in final_df.columns] + [col for col in final_df.columns if col not in final_columns]
final_df3 = final_df[final_columns].copy()

print(final_df3.head())  # Display the first few rows to verify the structure

Initializing StandardMetricsCalculator...
...Rouge object initialized.
                                          instruction  \
24  What is the last festival celebrated in Tihar ...   
26  Based on this paragraph, how many younger sibl...   
15    Who coined the phrase "Bike-shedding" and when?   
28  Given a reference text about Jen Shah, what is...   
20  Given this reference text about the consumptio...   

                                              context  \
24  Lakshmi Puja is celebrated as a part of Tihar,...   
26  Kennedy was born on February 22, 1932, at St. ...   
15  The law of triviality is C. Northcote Parkinso...   
28  Shah starred on the reality television series ...   
20  The United States is a leading exporter of pea...   

                                             response  \
24  The festival lasts for 5 days and Bhai Tika is...   
26  Based on this paragraph, Kennedy had no younge...   
15  The phrase "bike-shedding" was introduced in 1...   
28  Jen Shah ca

In [29]:
final_df2.head()

,instruction,context,response,predicted_response,accuracy,f1_score,bleu_score,rouge_l,rouge_3,bert_score,keyword_overlap,length_ratio
24,What is the last festival celebrated in Tihar ...,"Lakshmi Puja is celebrated as a part of Tihar,...",The festival lasts for 5 days and Bhai Tika is...,Lakshmi Puja is celebrated on the third day of...,0,0.181818,5.273843e-155,0.121212,0.000000,0.567486,0.200000,1.846154
26,"Based on this paragraph, how many younger sibl...","Kennedy was born on February 22, 1932, at St. ...","Based on this paragraph, Kennedy had no younge...",Kennedy was named after his father's assistant...,0,0.127660,4.430915e-155,0.085106,0.000000,0.502016,0.166667,2.000000
15,"Who coined the phrase ""Bike-shedding"" and when?",The law of triviality is C. Northcote Parkinso...,"The phrase ""bike-shedding"" was introduced in 1...","The law of triviality, also known as the bike-...",0,0.047619,2.274773e-155,0.047059,0.000000,0.432627,0.142857,7.818182
28,"Given a reference text about Jen Shah, what is...",Shah starred on the reality television series ...,Jen Shah came into the spotlight in November 2...,"Jen Shah, a star of The Real Housewives of Sal...",0,0.551724,2.660100e-01,0.511111,0.192308,0.730172,0.653846,0.573529
20,Given this reference text about the consumptio...,The United States is a leading exporter of pea...,National Peanut Butter Day takes place on Janu...,The United States is a major exporter and cons...,0,0.148148,3.515896e-155,0.140351,0.000000,0.451536,0.200000,4.153846


In [30]:
hallu_deepeval_df

,instruction,context,response,predicted_response,faithfulness_score,relevancy_score,hallucination_score
0,What is the last festival celebrated in Tihar ...,"Lakshmi Puja is celebrated as a part of Tihar,...",The festival lasts for 5 days and Bhai Tika is...,Lakshmi Puja is celebrated on the third day of...,0.666667,0.000000,0.0
1,"Based on this paragraph, how many younger sibl...","Kennedy was born on February 22, 1932, at St. ...","Based on this paragraph, Kennedy had no younge...",Kennedy was named after his father's assistant...,1.000000,0.000000,0.0
2,"Who coined the phrase ""Bike-shedding"" and when?",The law of triviality is C. Northcote Parkinso...,"The phrase ""bike-shedding"" was introduced in 1...","The law of triviality, also known as the bike-...",1.000000,0.000000,0.0
3,"Given a reference text about Jen Shah, what is...",Shah starred on the reality television series ...,Jen Shah came into the spotlight in November 2...,"Jen Shah, a star of The Real Housewives of Sal...",1.000000,0.800000,0.0
4,Given this reference text about the consumptio...,The United States is a leading exporter of pea...,National Peanut Butter Day takes place on Janu...,The United States is a major exporter and cons...,1.000000,0.000000,0.0
5,What is wine?,Wine is an alcoholic drink typically made from...,Wine is an alcoholic drink typically made from...,"Wine is primarily made from fermented grapes, ...",1.000000,1.000000,0.0
6,Who tends to participates in hackathons?,"A hackathon (also known as a hack day, hackfes...",Computer programmers and others involved in so...,A hackathon is an event where participants eng...,1.000000,0.428571,0.0
7,Given a reference text about Run Towards the D...,Run Towards the Danger is a 2022 Canadian essa...,Six essays are part of the Run Towards the Dan...,"""Run Towards the Danger"" is a 2022 essay colle...",1.000000,0.000000,0.0
8,What types of problems in government could AI ...,The potential uses of AI in government are wid...,The potential uses of AI in government are wid...,AI can be effectively applied in government op...,1.000000,1.000000,0.0
9,Which American president spoke Dutch as their ...,Van Buren received a basic education at the vi...,President Van Buren spoke Dutch as his first l...,"Martin Van Buren, the only U.S. president whos...",1.000000,0.285714,0.0


In [33]:
final_df2.to_csv('final_df2.csv', index=False)

In [ ]:
new_df = df.tail(15)